In [1]:
%cd /Users/vt2113/DonorAI/lvh/lvh
!pwd

/Users/vt2113/DonorAI/lvh/lvh
/Users/vt2113/DonorAI/lvh/lvh


In [4]:
import argparse
from operator import mod
from typing import Union
from argparse import ArgumentParser
import cv2
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import torch
from pathlib import Path
from tqdm import tqdm
from scipy.signal import find_peaks
import shutil
import sys
from threading import Thread, Lock

from utils import BoolAction, get_clip_dims, read_clip, get_systole_diastole, get_lens_np, get_points_np
from utils import get_angles_np, get_pred_measurements, overlay_preds, model_paths
from models import PlaxModel as Model
from time import time
plt_thread_lock = Lock()
import re
import os, os.path
from os.path import splitext,isfile,join
from os import listdir
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import csv


In [5]:
destinationFolder = "/Users/vt2113/Desktop/DICOM #2/LVHOutput"
inputFolder = '/Users/vt2113/Desktop/DICOM #2/LVH Data #2'
AVIFolder = '/Users/vt2113/Desktop/DICOM #2/LVHAVI'


In [4]:
def mask(output):

    dim1 = output.shape[0]
    dim2 = output.shape[1]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dim2), np.arange(dim1))
    

    mask = ((m1+m2)>(int(dim2/2) +int(dim2/12))) 
    # mask = m2> dim1/dim2*(-m1+int(dim2/2)) +int(dim2/10)
    mask *=  ((m1-m2)<(int(dim2/2) -int(dim2/12)))
    # mask *= -m2< dim1/dim2*(-m1+int(dim2/2)) -int(dim2/10)
    
    mask = np.reshape(mask, (dim1, dim2)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage

In [8]:
def makeVideo(fileToProcess, destinationFolder):

    fileName = fileToProcess.split('/')[-1] #\\ if windows, / if on mac or sherlock
                                             #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()

    if not os.path.exists(os.path.join(destinationFolder,fileName[:-4]+'.avi')):
        print(os.path.join(destinationFolder,fileName[:-4]+'.avi'))

        dataset = dicom.dcmread(fileToProcess, force=True)
        testarray = dataset.pixel_array
        # print(testarray.shape)
        if len(testarray.shape)!=4:
            return 

#         frame0 = testarray[0]
#         plt.imshow(frame0[:,:,2])
#         mean = np.mean(frame0, axis=1)[:,0]
# #         mean = np.mean(mean, axis=1)
# #         plt.plot(mean)
#         try:
#             yCrop = np.where(mean<1)[0][0]
#             testarray = testarray[:, yCrop:, :, :]
#         except:
#             pass

        # bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
        # if bias>0:
        #     if testarray.shape[1] < testarray.shape[2]:
        #         testarray = testarray[:, :, bias:-bias, :]
        #     else:
        #         testarray = testarray[:, bias:-bias, :, :]


        # print(testarray.shape)
        frames,height,width,channels = testarray.shape

        fps = 30

        try:
            fps = dataset[(0x18, 0x40)].value
        except:
            print("couldn't find frame rate, default to 30")

        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
        video_filename = os.path.join(destinationFolder, fileName[:-4] + '.avi')
        out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)


        for i in range(frames):

            outputA = testarray[i,:,:,0]
            smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

#             # Resize image
            output = cv2.resize(outputA, cropSize, interpolation = cv2.INTER_CUBIC)
            finaloutput = mask(output)


            finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
            out.write(finaloutput)

        out.release()

    else:
        print(fileName,"hasAlreadyBeenProcessed")

In [11]:
def extract_pixel(inputFolder,destination):
    onlyfiles = [f for f in listdir(inputFolder) if (isfile(join(inputFolder, f)) and f[-4:] ==".dcm")]
    fnames = [f.split('/')[-1] for f in onlyfiles]
    x_dim = []
    y_dim = []
    x = []
    y = []
    original_shape = []
    for f in tqdm(onlyfiles):
        dataset = dicom.dcmread(os.path.join(inputFolder,f), force=True)
        try:
            x_dim.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x6024)].value)
        except:
            x_dim.append(None)
        try:
            y_dim.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x6026)].value)
        except:
            y_dim.append(None)
        try:
            x.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x602c)].value)
        except:
            x.append(None)
        try:
            y.append(dataset[(0x18, 0x6011)].value[0][(0x18, 0x602e)].value)
        except:
            y.append(None)
        original_shape.append(dataset.pixel_array.shape)
        
        
    df = pd.DataFrame({
        "fname":fnames,
        "x_unit":x_dim,
        "y_unit":y_dim,
        "delta_x":x,
        "delta_y":y,
        "Original_dim":original_shape
    })
    df.to_csv(destination+os.path.sep+'meta_Data.csv')
    
    
    

In [12]:
## Convert to AVI 
cropSize = (640,480)
onlyfiles = [f for f in listdir(inputFolder) if isfile(join(inputFolder, f))]
for f in onlyfiles:
    makeVideo(os.path.join(inputFolder,f),AVIFolder)    

extract_pixel(inputFolder,destinationFolder)


LVH-OFF22-035.dcm hasAlreadyBeenProcessed
LVH-OFF22-021.dcm hasAlreadyBeenProcessed
LVH-OHT21-015.dcm hasAlreadyBeenProcessed
LVH-OHT21-014.dcm hasAlreadyBeenProcessed
LVH-OFF22-034.dcm hasAlreadyBeenProcessed
LVH-OFF22-022.dcm hasAlreadyBeenProcessed
/Users/vt2113/Desktop/DICOM #2/LVHAVI/LVH-OFF22-036.avi
LVH-OHT21-016.dcm hasAlreadyBeenProcessed
LVH-OHT21-017.dcm hasAlreadyBeenProcessed
LVH-OHT21-003.dcm hasAlreadyBeenProcessed
LVH-OFF22-037.dcm hasAlreadyBeenProcessed
LVH-OFF22-023.dcm hasAlreadyBeenProcessed
LVH-OFF22-027.dcm hasAlreadyBeenProcessed
LVH-OFF22-033.dcm hasAlreadyBeenProcessed
LVH-OHT21-007.dcm hasAlreadyBeenProcessed
LVH-OHT21-012.dcm hasAlreadyBeenProcessed
LVH-OHT21-006.dcm hasAlreadyBeenProcessed
LVH-OFF22-032.dcm hasAlreadyBeenProcessed
LVH-OFF22-026.dcm hasAlreadyBeenProcessed
LVH-OFF22-030.dcm hasAlreadyBeenProcessed
LVH-OFF22-024.dcm hasAlreadyBeenProcessed
LVH-OHT21-038.dcm hasAlreadyBeenProcessed
LVH-OHT21-005.dcm hasAlreadyBeenProcessed
/Users/vt2113/Deskto

  0%|                                                   | 0/103 [00:00<?, ?it/s]

LVH-OHT21-046.dcm hasAlreadyBeenProcessed
LVH-OHT21-047.dcm hasAlreadyBeenProcessed
LVH-OHT21-053.dcm hasAlreadyBeenProcessed
LVH-OFF22-028.dcm hasAlreadyBeenProcessed
LVH-OHT21-034.dcm hasAlreadyBeenProcessed
LVH-OHT21-021.dcm hasAlreadyBeenProcessed
LVH-OHT21-035.dcm hasAlreadyBeenProcessed
LVH-OFF22-029.dcm hasAlreadyBeenProcessed
LVH-OHT21-023.dcm hasAlreadyBeenProcessed
LVH-OHT21-036.dcm hasAlreadyBeenProcessed
LVH-OHT21-022.dcm hasAlreadyBeenProcessed
LVH-OHT21-013-1.dcm hasAlreadyBeenProcessed
LVH-OHT21-032.dcm hasAlreadyBeenProcessed
LVH-OHT21-033.dcm hasAlreadyBeenProcessed
LVH-OHT21-027.dcm hasAlreadyBeenProcessed
LVH-OFF22-039.dcm hasAlreadyBeenProcessed
/Users/vt2113/Desktop/DICOM #2/LVHAVI/LVH-OHT21-013-2.avi
LVH-OHT21-019.dcm hasAlreadyBeenProcessed
LVH-OHT21-031.dcm hasAlreadyBeenProcessed
LVH-OHT21-025.dcm hasAlreadyBeenProcessed
LVH-OHT21-024.dcm hasAlreadyBeenProcessed
LVH-OHT21-018.dcm hasAlreadyBeenProcessed
LVH-OFF22-038.dcm hasAlreadyBeenProcessed


100%|█████████████████████████████████████████| 103/103 [04:16<00:00,  2.49s/it]


In [6]:
model_path = model_paths['plax']
device='cpu'

In [7]:
model = Model().eval().to(device)

In [8]:
model.load_state_dict(torch.load(model_path,map_location=device))


<All keys matched successfully>

In [12]:
paths = Path(AVIFolder)
paths = list(paths.iterdir())
print(paths)


[PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-69.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-55.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-41.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-82.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OFF22-020.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OFF22-008.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OFF22-009.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-40.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-54.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-68.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-56.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-80.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-57.avi'), PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-43.avi'), Po

In [30]:
paths[18]

PosixPath('/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-85.avi')

In [31]:
(n, w_all, h_all), fnames = get_clip_dims(paths)

In [32]:
frame_map = pd.DataFrame({
            'frame': np.concatenate([np.arange(ni) for ni in n]),
            'path': np.concatenate([np.array([str(p)] * ni, dtype=np.object) for ni, p in zip(n, paths)]),
        })

/var/folders/hf/_4l7f46s7sx36_h8zwshkn_41s8kcx/T/ipykernel_83255/1172152414.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'path': np.concatenate([np.array([str(p)] * ni, dtype=np.object) for ni, p in zip(n, paths)]),


In [33]:
batch_size=10
h=480
w=640
channels_in=3
channels_out=4
verbose=True


In [34]:
clips = dict()  # clips currently being processed
batch = np.zeros((batch_size, h, w, channels_in))


In [35]:
def run_model_np(x: np.ndarray) -> np.ndarray:
    """Run inference on a numpy array video.
    Args:
        x (np.ndarray): shape=(n, h, w, 3)
    Returns:
        (np.ndarray): shape=(n, h, w, 4). Raw model predictions.
    """
    input_tensor = torch.tensor(x,dtype = torch.float32,device = device, requires_grad = False).permute(0,3,1,2)/255
    with torch.no_grad():
        preds_tensor = model(input_tensor)
        preds_tensor = preds_tensor.permute(0,2,3,1)
    input_tensor = input_tensor.detach().numpy()
    preds = preds_tensor.detach().numpy()
    return preds

In [ ]:
for si in tqdm(range(0, len(frame_map), batch_size)) if verbose else range(0, len(frame_map), batch_size):
        start = time() 
        # Get batch files
        batch_map = frame_map.iloc[si:min(si + batch_size, len(frame_map))]
        batch_paths = batch_map['path'].unique()
        l = list(clips.items())
        # print("Get batch files")
        # print(time()-start)
        start = time() 
        # Check if inference has finished for all current clips
        # and yield results for any finished.
#         for k, v in l:
#             if k not in batch_paths:
#                 clips.pop(k)
        # print("Check inference")
        # print(time()-start)
        # start = time() 

        # Generate batch
        for p in batch_paths:
            # print('Reading again')
            if p not in clips:
                # print('Reading again 2')
                c = read_clip(p, res=(w, h))
                clips[p] = (c, np.zeros((len(c), h, w, channels_out), dtype=np.float))
            batch[:len(batch_map)][batch_map['path'] == p] = clips[p][0][batch_map[batch_map['path'] == p]['frame']]
        
        # print("Geenerate Batch")
        # print(time()-start)
        start = time() 

        # Run inference and set results
        preds = run_model_np(batch[:len(batch_map)])
        # print("Run model")
        # print(time()-start)
        start = time()
        
        for p in batch_paths:
            clips[p][1][batch_map[batch_map['path'] == p]['frame']] = preds[batch_map['path'] == p]
        
        # print("Run Inference")
        # print(time()-start)
    

  0%|                                                   | 0/628 [00:00<?, ?it/s]/var/folders/hf/_4l7f46s7sx36_h8zwshkn_41s8kcx/T/ipykernel_83255/2351262209.py:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  clips[p] = (c, np.zeros((len(c), h, w, channels_out), dtype=np.float))
 90%|██████████████████████████████████▎   | 568/628 [10:21:54<58:52, 58.87s/it]

In [80]:
clips['/Users/vt2113/Desktop/OHT DICOMS/LVHAVI/LVH-OHT22-69.avi'][1].shape


(43, 480, 640, 4)

In [6]:
def save_preds(
            p: Union[str, Path], fn: str, clip: np.ndarray, preds: np.ndarray, 
            csv=True, avi=True, plot=True, npy=False, angle_threshold=30
        ) -> None:
    """Save results for model predictions. All results are saved to a new directory within the 
    output path with the name [fn]. Frame-by-frame analysis to predict ES and ED is used and 
    results are saved to [fn].csv.

    Args:
        p (Union[str, Path]): output path to save to
        fn (str): output filename
        clip (np.ndarray): shape=(n, h, w, 3), input clip used to run inference
        preds (np.ndarray): shape=(n, h, w, 4), model predictions from input clip
        csv (bool, optional): Save frame-by-frame results to .csv. Defaults to True.
        avi (bool, optional): Save prediction animation as .avi. Defaults to True.
        plot (bool, optional): Save measurement vs time plot as .png. Defaults to True.
        npy (bool, optional): Save raw predictions as numpy .npy. Defaults to False.
        angle_threshold (int, optional): Angle between measurement lines above which are 
            considered 'bad'. Defaults to 30.
    """

    # Create folder
    folder_name = fn.replace('.avi', '').replace('.', '_')
    inf_path = p / folder_name
    if not inf_path.exists():
        inf_path.mkdir()
    
    # Save raw predictions as .npy
    if npy:
        print(f'Saving npy {inf_path / (folder_name)}')
        np.save(inf_path / (folder_name + '.npy'), preds)
    pred_pts, pred_lens, sys_i, dia_i, angles = get_pred_measurements(preds)

    # Save predicted points to .csv
    if csv:
        phase = np.array([''] * len(pred_pts), dtype=object)
        phase[sys_i] = 'ES'
        phase[dia_i] = 'ED'
        df = pd.DataFrame({
            'frame': np.arange(len(pred_pts)),
            'X1': pred_pts[:, 0, 0],
            'Y1': pred_pts[:, 0, 1],
            'X2': pred_pts[:, 1, 0],
            'Y2': pred_pts[:, 1, 1],
            'X3': pred_pts[:, 2, 0],
            'Y3': pred_pts[:, 2, 1],
            'X4': pred_pts[:, 3, 0],
            'Y4': pred_pts[:, 3, 1],
            'LVPW': pred_lens[:, 0],
            'LVID': pred_lens[:, 1],
            'IVS': pred_lens[:, 2],
            'predicted_phase': phase,
            'LVPW_LVID_angle': angles[:, 0],
            'LVID_IVS_angle': angles[:, 1],
            'bad_angle': (abs(angles[:, 0]) > angle_threshold) | (abs(angles[:, 1]) > angle_threshold)
        })
        df.set_index('frame')
        df.to_csv(inf_path / (folder_name + '.csv'))

    # Save an animation of the predictions overlayed on the cropped video as .avi
    if avi:
        with plt_thread_lock:
            # make_animation(inf_path / (folder_name + '.avi'), clip, preds, pred_pts, pred_lens, sys_i, dia_i)
            make_animation_cv2(inf_path / (folder_name + '.avi'), clip, preds, pred_pts)
    
    # Save a plot of measurements vs time for whole clip
    if plot:
        make_plot(inf_path / (folder_name + '.png'), folder_name, pred_lens, sys_i, dia_i)

def make_animation(
            save_path: Union[Path, str], clip: np.ndarray, preds: np.ndarray, 
            pred_pts: np.ndarray, pred_lens: np.ndarray, sys_i, dia_i, 
            figsize=(12, 12), units='PX', fps=50
        ) -> None:
    
    """Save animation of predictions using matplotlib.

    Args:
        save_path (Union[Path, str]): Location to save animation .avi to
        clip (np.ndarray): shape=(n, h, w, 3). Input video used for inference.
        preds (np.ndarray): shape=(n, h, w, 4). Raw model predictions.
        pred_pts (np.ndarray): shape=(n, 4, 2). Predicted points for measurements.
        pred_lens (np.ndarray): shape=(n, 3). Predicted measurement values [LVPW, LVID, IVS]
        sys_i (array-like): indices predicted to be end systole
        dia_i (array-like): indices predicted to be end diastole
        figsize (tuple, optional): plt figure size. Defaults to (12, 12).
        units (str, optional): Units to show on plot. Defaults to 'PX'.
        fps (int, optional): Frame rate so save animation in. Defaults to 50.
    """
    
    # Ensure Path object
    if isinstance(save_path, str):
        save_path = Path(save_path)
    
    # Setup figure layout and static plot
    grid = plt.GridSpec(4, 1)
    fig = plt.figure(0, figsize=figsize)
    ax1 = fig.add_subplot(grid[3:, 0])
    ax2 = fig.add_subplot(grid[:3, 0])
    for l, n in zip(pred_lens.T, ['LVPW', 'LVID', 'IVS']):
        ax1.plot(l, label=n)
    l1, = ax1.plot([0, 0, 0], pred_lens[0], 'ro')
    ax1.legend()
    ax1.set_xlabel('Frame')
    ax1.set_ylabel(f'Measurement [{units}]')
    ax1.vlines(sys_i, pred_lens.min(), pred_lens.max(), linestyles='dashed', colors='b', label='Systole')
    ax1.vlines(dia_i, pred_lens.min(), pred_lens.max(), linestyles='dashed', colors='g', label='Diastole')
    im = ax2.imshow(overlay_preds(preds[0], clip[0] / 255))
    l2, = ax2.plot(*pred_pts[0].T, 'C1o-')
    ax2.set_title(save_path.name)

    # Modifies plot for each frame of animation
    def animate(i):
        im.set_data(overlay_preds(preds[i], clip[i] / 255))
        l1.set_data([i, i , i], pred_lens[i])
        l2.set_data(*pred_pts[i].T)

    # Save animation
    ani = animation.FuncAnimation(fig, animate, frames=len(clip), interval=1000 / fps)
    writer = animation.FFMpegWriter(fps)
    ani.save(save_path, writer)

    del fig

def make_plot(
            save_path: Union[Path, str], title: str, pred_lens: np.ndarray, 
            sys_i, dia_i, figsize=(8, 6)
        ) -> None:
    """Save a plot showing measurement values over time.

    Args:
        save_path (Union[Path, str]): .png path to save plot to.
        title (str): Plot title.
        pred_lens (np.ndarray): shape=(n, 3). Predicted measurements [LVPW, LVID, IVS]
        sys_i (array-like): indices predicted to be end systole
        dia_i (array-like): indices predicted to be end diastole
        figsize (tuple, optional): plt figure size. Defaults to (8, 6).
    """

    plt.figure(1, figsize=figsize)
    plt.clf()
    for l, n in zip(pred_lens.T, ['LVPW', 'LVID', 'IVS']):
        plt.plot(l, label=n)
    plt.plot(sys_i, pred_lens[sys_i], 'r+')
    plt.plot(dia_i, pred_lens[dia_i], 'rx')
    plt.plot([], [], 'rx', label='Diastole')
    plt.plot([], [], 'r+', label='Systole')
    plt.title(title)
    plt.xlabel('Frame')
    plt.ylabel('Measurement [px]')
    plt.legend()
    plt.savefig(save_path)


def make_animation_cv2(
            save_path: Union[Path, str], clip: np.ndarray, preds: np.ndarray, pred_pts: np.ndarray, 
            fps=30.0, line_color=(1, 1, 0), point_color=(1, 0.5, 0), linewidth=2, markersize=4
        ) -> None:
    """Creates an animation with predictions overlayed on top of the clip.

    Args:
        save_path (Union[Path, str]): .avi path to save animation to.
        clip (np.ndarray): shape=(n, h, w, 3). Input video used for inference.
        preds (np.ndarray): shape=(n, h, w, 4). Raw model predictions.
        pred_pts (np.ndarray): shape=(n, 4, 2). Predicted points for measurements.
        fps (float, optional): Animation frame rate. Defaults to 30.0.
        line_color (tuple, optional): Color of measurement lines. Defaults to (1, 1, 0).
        point_color (tuple, optional): Color of measurement endpoints. Defaults to (1, 0.5, 0).
        linewidth (int, optional): Width of measurement lines. Defaults to 2.
        markersize (int, optional): Size of measurement endpoints. Defaults to 4.
    """
    out = cv2.VideoWriter(str(save_path), cv2.VideoWriter_fourcc(*'MJPG'), fps, (clip.shape[2], clip.shape[1]))
    for frame, pred, line in zip(clip, preds, pred_pts):
        
        # Overlay raw predictions
        img = overlay_preds(pred, frame / 255)
        if not np.isnan(line).any():
            line = line.round().astype(int)
        
            # Draw measurement
            for pt0, pt1 in zip(line[:-1], line[1:]):
                img = cv2.line(img, tuple(pt0), tuple(pt1), line_color, linewidth)
            for pt in line:
                img = cv2.circle(img, tuple(pt), radius=markersize, color=point_color, thickness=-1)
        
        # Write to file
        img = (img * 255).astype(np.uint8)
        out.write(img[:, :, ::-1])
    
    # Close file
    out.release()

In [ ]:
for fn,(raw,preds) in clips.items():
    fn = fn.split('/')[-1]
    save_preds(Path(destinationFolder),fn,raw,preds,csv=True, avi=True, plot=True,npy=True)


In [21]:
test = {'1':(0,3)}

In [18]:
def convert_px_to_cm(input_folder):
    resize = (480,640)
    onlyfiles = [f for f in listdir(input_folder) if not isfile(join(input_folder,f))]
    onlyfiles.pop(24)
    df_meta = pd.read_csv(os.path.join(input_folder,'meta_Data.csv'))
    i = 0
    for f in tqdm(onlyfiles):
        df = pd.read_csv(os.path.join('/Users/vt2113/Desktop/DICOM #2/LVHOutput_mod',f,f+'.csv'))
        df_now = df_meta[df_meta.fname==f+".dcm"]
        dims = eval(list(df_now.Original_dim)[0])
        x_scale = abs(df_now.delta_x * dims[1]/resize[0]).to_numpy()[0]
        y_scale = abs(df_now.delta_y * dims[2]/resize[1]).to_numpy()[0]
        pts = np.array([[df.X1.to_numpy() *x_scale,df.Y1.to_numpy() *y_scale],
                        [df.X2.to_numpy() *x_scale,df.Y2.to_numpy() *y_scale],
                        [df.X3.to_numpy() *x_scale,df.Y3.to_numpy() *y_scale],
                        [df.X4.to_numpy() *x_scale,df.Y4.to_numpy() *y_scale]])
        
        df['X1_cm'] = df.X1 * x_scale
        df['X2_cm'] = df.X2 * x_scale
        df['X3_cm'] = df.X3 * x_scale
        df['X4_cm'] = df.X4 * x_scale
        df['Y1_cm'] = df.Y1 * y_scale
        df['Y2_cm'] = df.Y2 * y_scale
        df['Y3_cm'] = df.Y3 * y_scale
        df['Y4_cm'] = df.Y4 * y_scale
        pts = np.moveaxis(pts,-1,0)
        pred_lens = get_lens_np(pts)
        df['LVPW_cm'] =  np.squeeze(pred_lens[:, 0])
        df['LVID_cm'] = np.squeeze(pred_lens[:, 1])
    
        df['IVS_cm'] =  np.squeeze(pred_lens[:, 2])
        df.to_csv(os.path.join(input_folder,f,f+'_cm.csv'))
        



In [19]:
convert_px_to_cm(destinationFolder)


100%|███████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 34.87it/s]


In [11]:
onlyfiles = [f for f in listdir(destinationFolder) if not isfile(join(destinationFolder,f))]
for f in tqdm(onlyfiles[25:]):
    preds = np.load(os.path.join(destinationFolder,f,f+'.npy'))
    clip = read_clip(os.path.join(AVIFolder,f+'.avi'))
    try:
        save_preds(Path('/Users/vt2113/Desktop/DICOM #2/LVHOutput_mod'),f,clip,preds,csv=True,avi=False,plot=False,npy=False)
    except:
        print(f"Skipped {f}")

100%|███████████████████████████████████████████████████████████| 30/30 [02:47<00:00,  5.59s/it]


preds.shape

In [28]:
onlyfilesAVI = [f[:-4] for f in listdir(AVIFolder) if isfile(join(AVIFolder,f))]
onlyfilesOut = [f for f in listdir(destinationFolder) if not isfile(join(destinationFolder,f))]
s = set(onlyfilesOut)
left_files = [f for f in onlyfilesAVI if f not in s]
print(left_files)


['LVH-OHT21-072', 'LVH-OHT21-067', 'LVH-OFF22-053', 'LVH-OFF22-047', 'LVH-OHT23-002', 'LVH-OHT21-042', 'LVH-OHT21-056', 'LVH-OHT21-043', 'LVH-OFF22-049', 'LVH-OHT21-055', 'LVH-OHT21-054', 'LVH-OFF22-048', 'LVH-OHT23-009', 'LVH-OHT21-050', 'LVH-OHT21-044', 'LVH-OHT21-079', 'LVH-OHT21-045', 'LVH-OHT21-051', 'LVH-OHT23-008', 'LVH-OHT21-047', 'LVH-OHT21-053', 'LVH-OHT21-046', 'LVH-OFF22-029', 'LVH-OHT21-021', 'LVH-OHT21-035', 'LVH-OHT21-034', 'LVH-OFF22-028', 'LVH-OHT21-036', 'LVH-OHT21-022', 'LVH-OHT21-023', 'LVH-OHT21-033', 'LVH-OHT21-027', 'LVH-OHT21-032', 'LVH-OHT21-013-1', 'LVH-OFF22-038', 'LVH-OHT21-024', 'LVH-OHT21-018', 'LVH-OHT21-019', 'LVH-OHT21-031', 'LVH-OHT21-025', 'LVH-OFF22-039']


In [29]:
import shutil
import os
trg = '/Users/vt2113/Desktop/DICOM #2/AVISubset'
for fname in left_files:
    # copying the files to the
    # destination directory
    shutil.copy2(os.path.join(AVIFolder,fname+'.avi'), trg)